In [1]:
import gensim
import json
import re
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
import pyLDAvis.gensim 

morph = MorphAnalyzer()

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import collections

## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками**.

1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) заново обучите LDA c теми же параметрами (параметрами самой лучшей модели, заново перебирать не нужно);

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

## Данные

In [3]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def clean(words):
    clean = [morph.parse(word)[0].normal_form for word in words if len(set(word)) > 1]
    return clean

Возьмем 4 тыс статьи с Хабра. Это мало для хорошей тематической модели, но иначе у нас просто ничего не обучится за семинар.

In [4]:
texts = open('habr_texts.txt', encoding='utf-8').read().splitlines()

In [5]:
from collections import Counter

In [39]:
f = open('habr_texts.vw', 'w',  encoding='utf-8')

for i, text in enumerate(texts):
    c = Counter(text)
    doc = 'doc_'+ str(i) + ' '
    vw_text = ' '.join([x+':'+str(c[x]) for x in c])
    
    f.write(doc + vw_text  + '\n')
f.close()



In [157]:
#print(texts[1])

В текстах есть тэги. Потрем их. Ещё токенизируем самым простым способом и нормализуем Pymorphy.

In [41]:
texts = [clean(word_tokenize(text.lower())) for text in texts]

AttributeError: 'list' object has no attribute 'lower'

In [42]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.9) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [43]:
txt09 = print(ph[texts[0]])

['masstransit', 'это', 'open', 'source', 'библиотека', 'разработать', 'на', 'язык', 'для', '.net', 'платформа', 'упрощать', 'работа', 'шина', 'дать', 'который', 'использоваться', 'при', 'построение', 'распределенный', 'приложение', 'реализация', 'soa', 'service', 'oriented', 'architecture', 'качество', 'message', 'broker', 'мочь', 'выступать', 'rabbitmq', 'azure', 'service', 'bus', 'или', 'in-memory', 'менеджер', 'случай', 'in-memory', 'область', 'видимость', 'ограничиваться', 'процесс', 'который', 'проинициализировать', 'экземпляр', '.содержание', 'команда', 'событиякоманда', 'событие', 'контракт', 'сообщение', 'роутингexchange', 'формат', 'сообщение', 'консьюмер', 'consumer', 'конфигурация', 'контейнер', 'di', 'наблюдатель', 'observer', 'новое', 'masstransit', '3.0', 'заключение', 'опрос', 'какой', '.net', 'библиотека', 'использовать', 'вы', 'команда', 'событие', 'библиотека', 'заложить', 'основный', 'тип', 'сообщение', 'команда', 'события.команда', 'сигнализировать', 'необходимость'

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\gensim\models\phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


### Удаление стопслов

In [111]:
from nltk.corpus import stopwords
 
def no_stops(texts): 
    #deleting stop_words
    texts_new = []
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', 'к', 'на', 'ваш' 'the', 'if', 'gt', 'ip', 'rm', 'int', 'http','lt', 'of', 'in', 'at'])
    for text in texts:
        text_new = [word for word in text if ( word not in stop_words )]
        texts_new.append(text_new)
    return texts_new

In [112]:
texts_nostops = no_stops(texts)

### Тематическое моделирование в gensim

Для моделей нужно сделать словарь.

In [149]:
dictinary = gensim.corpora.Dictionary(texts_nostops)

In [150]:
dictinary.filter_extremes(no_above=0.4, no_below=40)
dictinary.compactify()

In [152]:
print(dictinary)

Dictionary(6194 unique tokens: ['тело', 'подойти', 'угол', 'оплачивать', 'нередко']...)


Преобразуем наши тексты в мешки слов. 

In [116]:
#corpus = [dictinary.doc2bow(text) for text in texts]
# если текстов много, то тут может быть генератор

In [153]:
tfidf = (gensim.models.TfidfModel(corpus, id2word=dictinary))    

In [154]:
corpus = tfidf[corpus]

In [155]:
lda = gensim.models.LdaMulticore(corpus, 70, id2word=dictinary, passes=10)

Посмотрим на топики.

### Без tfidf

In [135]:
besttopics = lda.print_topics()

В целом по-моему получилось довольно неплохо, поэтому выбрать было тяжело. Все темы, получившиеся после отработки модели бещ tfidf я сохранила в переменную besttopics.
Три из них, которые понравились мне больше всего, ниже:

#### Программрование

 (21,
  '0.017*"код" + 0.016*"язык" + 0.013*"функция" + 0.011*"тип" + 0.009*"объект" + 0.009*"класс" + 0.007*"метод" + 0.006*"swift" + 0.005*"реализация" + 0.005*"значение"')

#### Мобильные приложения

 (40,
  '0.096*"приложение" + 0.051*"android" + 0.023*"устройство" + 0.014*"мобильный" + 0.013*"google" + 0.011*"ios" + 0.010*"сообщение" + 0.009*"экран" + 0.009*"app" + 0.007*"ваш"')

#### Медицина


 (65,
  '0.015*"мозг" + 0.011*"глаз" + 0.008*"клетка" + 0.007*"пациент" + 0.007*"операция" + 0.005*"боль" + 0.005*"ген" + 0.005*"животное" + 0.005*"организм" + 0.004*"лазер"')

И все получившиеся после обучения лучшей модели без tfidf ниже

In [139]:
besttopics

[(68,
  '0.016*"пакет" + 0.010*"адрес" + 0.008*"сеть" + 0.007*"сетевой" + 0.007*"очередь" + 0.007*"файл" + 0.007*"маршрутизатор" + 0.006*"протокол" + 0.006*"маршрут" + 0.006*"команда"'),
 (28,
  '0.017*"товар" + 0.009*"элемент" + 0.009*"устройство" + 0.008*"питание" + 0.007*"батарея" + 0.007*"аккумулятор" + 0.006*"искажение" + 0.006*"оборудование" + 0.005*"характеристика" + 0.005*"100"'),
 (67,
  '0.026*"устройство" + 0.014*"кабель" + 0.014*"usb" + 0.010*"порт" + 0.009*"беспроводный" + 0.008*"php" + 0.007*"файл" + 0.007*"компьютер" + 0.007*"ноутбук" + 0.006*"oracle"'),
 (33,
  '0.013*"for" + 0.012*"значение" + 0.011*"массив" + 0.011*"код" + 0.009*"число" + 0.009*"тип" + 0.008*"цикл" + 0.008*"память" + 0.008*"метод" + 0.007*"double"'),
 (32,
  '0.039*"public" + 0.028*"new" + 0.026*"return" + 0.026*"string" + 0.021*"void" + 0.018*"private" + 0.017*"метод" + 0.015*"класс" + 0.014*"val" + 0.013*"null"'),
 (60,
  '0.054*"amp" + 0.013*"return" + 0.011*"char" + 0.011*"код" + 0.011*"функция" +

### С tfidf

In [156]:
tfidftopics = lda.print_topics()

In [162]:
tfidftopics

[(10,
  '0.009*"палитра" + 0.008*"твой" + 0.005*"фронтенд" + 0.005*"затрагивать" + 0.004*"адаптивный" + 0.004*"адаптация" + 0.003*"упрощать" + 0.003*"basic" + 0.003*"ограниченный" + 0.003*"разрешение"'),
 (44,
  '0.003*"одобрение" + 0.002*"context" + 0.001*"-e" + 0.001*"php" + 0.001*"this-" + 0.001*"vpn" + 0.001*"34" + 0.001*"option" + 0.001*"/**" + 0.001*"китайский"'),
 (24,
  '0.002*"атомный" + 0.001*"лёд" + 0.001*"const" + 0.000*"арифметический" + 0.000*"ruby" + 0.000*"export" + 0.000*"весы" + 0.000*"разработчик" + 0.000*"one" + 0.000*"переменный"'),
 (23,
  '0.040*"def" + 0.029*"val" + 0.014*"уравнение" + 0.011*"self" + 0.010*"теорема" + 0.008*"import" + 0.006*"генератор" + 0.006*"61" + 0.005*"none" + 0.005*"return"'),
 (20,
  '0.009*"up" + 0.001*"down" + 0.001*"-l" + 0.001*"curl" + 0.001*"файл" + 0.001*"станция" + 0.001*"папка" + 0.001*"-e" + 0.001*"-a" + 0.001*"приложение"'),
 (2,
  '0.014*"new" + 0.005*"day" + 0.005*"31" + 0.003*"when" + 0.003*"then" + 0.003*"values" + 0.002*"se

Кажется, что после добавления tfidf модель стала хуже - если без него каждая тема более или менее связная и можно без большого труда предположить, что имелось в виду, то во втором случае кажется, что больше чем в половине случаев вышла каша.
Узнаваемых тем, которые были и изменились я не вижу, но вижу знакомые слова, которые сочетаются очень странным способом. При этом перплексия стала ниже, а когерентность вообще не изменилась.

Ещё есть штука для визуализации.

In [163]:
#pyLDAvis.enable_notebook()

In [164]:
#pyLDAvis.gensim.prepare(lda, corpus, dictinary)

Можно посмотреть метрики.

In [121]:
import numpy as np

In [122]:
?lda.log_perplexity

In [123]:
lda.log_perplexity(corpus[:2000], total_docs=100)

-23.067781433972737

In [124]:
coherence_model_lda = gensim.models.CoherenceModel(model=lda, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [125]:
topics = []
for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
    topic = [word for word, _ in topic]
    topics.append(topic)

In [126]:
coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=dictinary, coherence='c_v')

In [127]:
coherence_model_lda.get_coherence()

0.5833270757337577

In [158]:
tfidfperp = lda.log_perplexity(corpus[:2000], total_docs=100)

In [159]:
tfidfcoher = coherence_model_lda.get_coherence()

In [160]:
tfidfperp

-62.63296381935896

In [161]:
tfidfcoher

0.5833270757337577

### Разложение матриц в sklearn

In [128]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd

Sklearn принимает на вход строки, поэтому склеим наши списки.

In [165]:
stexts = [' '.join(text) for text in texts]

Сделаем матрицу слова-документы с помощью TfidfVectorizer

In [180]:
vectorizer = TfidfVectorizer(max_features=25000, min_df=10, max_df=0.5, lowercase=False)
X = vectorizer.fit_transform(stexts)

C:\Users\Asus\Anaconda3\envs\mllecture\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Разложим её.

In [181]:
model = NMF(n_components=30)

In [182]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [183]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(30):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

Лучшая nmf модель ниже:

In [184]:
a = get_nmf_topics(model, 10)
a

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,человек,gt,игра,windows,сигнал,компания,приложение,public,космический,if,...,обучение,проект,файл,сайт,бот,книга,функция,камера,атака,робот
1,ты,lt,игрок,microsoft,звук,сотрудник,android,string,спутник,int,...,сеть,код,docker,пользователь,telegram,часы,объект,устройство,безопасность,ребёнок
2,мой,div,игровой,linux,усилитель,бизнес,мобильный,void,ракета,amp,...,нейросеть,задача,http,страница,сообщение,каковать,значение,смартфон,уязвимость,автомобиль
3,кто,class,играть,версия,частота,рынок,ios,new,орбита,lt,...,нейронный,язык,php,реклама,чат,порекомендовать,элемент,телефон,устройство,робототехника
4,вот,name,vr,azure,наушник,ит,google,this,марс,else,...,машинный,разработчик,сервер,клиент,телеграм,профессиональный,код,видео,злоумышленник,lego
5,ваш,this,персонаж,браузер,устройство,российский,устройство,return,аппарат,return,...,алгоритм,разработка,sudo,контент,bot,каким,тип,регистратор,защита,датчик
6,да,props,разработчик,net,искажение,продукт,app,private,земля,void,...,интеллект,команда,etc,письмо,api,предпочитать,строка,аккумулятор,пароль,машина
7,надо,html,steam,обновление,напряжение,крупный,пользователь,var,луна,null,...,изображение,тест,nginx,браузер,пользователь,слушать,метод,экран,вредоносный,беспилотный
8,дело,return,unity,visual,звуковой,директор,play,class,станция,char,...,лекция,тестирование,скрипт,сервис,мессенджер,бумажный,класс,карта,ключ,движение
9,хотеть,value,геймплей,studio,ток,клиент,разработчик,класс,наса,std,...,искусственный,программист,usr,домен,канал,менеджер,блок,дисплей,доступ,дрон


Кажется, что в отличии от тем, полученных с LDA эта модель дает описания именно статей, а в lda cкорее получилить топики - сферы и общее направление рассуждений.
То есь, в LDA на мой взгляд темы более общие, а в NMF более конкретные. Хотя они во многом похожи и некоторые даже кажутся совпадают, например и там и там выделилась тема про космос, про игры, про мобильные приложения и т.п.